In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from pycaret.regression import *

In [2]:
data = pd.read_csv('./filled_data.csv', index_col=0)
model_data = data.copy()
data.head()

,cena,ulica,pokoje,powierzchnia,piętro,liczba pięter,stan wykonczenia,balkon/ogrod/taras,miejsce parkingowe,rok budowy
0,906780.0,Zamkowa,5,88.90,1,1,Do odświeżenia,True,True,1969.0
1,649000.0,Zamkowa,2,38.65,1,6,Do odświeżenia,True,False,1969.0
2,824000.0,Nowowiejska,2,66.71,4,5,Gotowe do wprowadzenia,True,False,1969.0
3,519800.0,Rudzka,2,35.00,5,10,Gotowe do wprowadzenia,False,True,1969.0
4,639000.0,Brzeska,2,39.50,1,4,Gotowe do wprowadzenia,False,False,1969.0


In [3]:
model_data['stan wykonczenia'] = model_data['stan wykonczenia'].map({'Do remontu': 0, 'Do odświeżenia': 1, 'Gotowe do wprowadzenia': 2})
label_encoder = LabelEncoder()
model_data['ulica'] = label_encoder.fit_transform(model_data['ulica'])
model_data

,cena,ulica,pokoje,powierzchnia,piętro,liczba pięter,stan wykonczenia,balkon/ogrod/taras,miejsce parkingowe,rok budowy
0,906780.0,636,5,88.900000,1,1,1,True,True,1969.0
1,649000.0,636,2,38.650000,1,6,1,True,False,1969.0
2,824000.0,375,2,66.710000,4,5,2,True,False,1969.0
3,519800.0,478,2,35.000000,5,10,2,False,True,1969.0
4,639000.0,53,2,39.500000,1,4,2,False,False,1969.0
...,...,...,...,...,...,...,...,...,...,...
9485,964189.0,661,2,44.340000,10,23,0,True,True,1969.0
9486,664376.0,150,3,60.450001,1,2,1,True,True,1969.0
9487,797440.0,377,3,68.599998,2,4,1,True,True,1969.0
9488,1027176.0,661,3,52.000000,5,23,0,True,True,1969.0


In [4]:
randomized_data = model_data.sample(frac=1, random_state=42)

randomized_data.reset_index(drop=True, inplace=True)

train_size = int(0.75 * len(randomized_data))
train_data = randomized_data[:train_size]
test_data = randomized_data[train_size:]
train_data

,cena,ulica,pokoje,powierzchnia,piętro,liczba pięter,stan wykonczenia,balkon/ogrod/taras,miejsce parkingowe,rok budowy
0,899000.0,384,4,88.50,0,0,1,True,False,1969.0
1,850000.0,699,3,76.36,1,3,2,True,False,1969.0
2,539000.0,182,4,67.50,1,1,0,True,True,1969.0
3,620000.0,125,1,29.62,4,7,2,False,False,1969.0
4,806000.0,315,3,63.26,4,6,0,True,True,1969.0
...,...,...,...,...,...,...,...,...,...,...
7112,542459.5,254,3,51.91,2,2,1,True,True,1969.0
7113,569000.0,478,2,61.70,1,3,0,False,False,1969.0
7114,692724.0,257,3,57.21,3,4,1,True,True,1969.0
7115,869000.0,125,3,60.61,1,4,0,True,False,1969.0


In [5]:
exp1 = setup(data=train_data, target='cena')
mod = create_model('xgboost', data = train_data, target = 'cena')
tuned_model = tune_model(mod)
tuned_model.get_params


,Description,Value
0,Session id,4425
1,Target,cena
2,Target type,Regression
3,Original data shape,"(7117, 10)"
4,Transformed data shape,"(7117, 10)"
5,Transformed train set shape,"(4981, 10)"
6,Transformed test set shape,"(2136, 10)"
7,Numeric features,7
8,Preprocess,True
9,Imputation type,simple


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,78716.5938,24171915264.0000,155473.2031,0.6384,0.1687,0.1091
1,85128.4141,27835942912.0000,166841.0781,0.7176,0.1647,0.1102
2,87504.1094,34595352576.0000,185998.2656,0.7154,0.1611,0.0991
3,80642.8438,28842027008.0000,169829.4062,0.8125,0.1609,0.1064
4,88361.2188,28999499776.0000,170292.3906,0.8399,0.1630,0.1054
5,73949.5078,25162625024.0000,158627.3125,0.8277,0.1335,0.0872
6,83991.7578,21209028608.0000,145633.2031,0.7698,0.1556,0.1077
7,99301.6562,129186365440.0000,359425.0625,0.5435,0.1760,0.1034
8,73368.7344,18179201024.0000,134830.2656,0.7600,0.1450,0.0952


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,77035.3438,22025097216.0000,148408.5469,0.6705,0.1747,0.1105
1,84081.8203,25542690816.0000,159820.8125,0.7409,0.1583,0.1091
2,85914.1641,31294330880.0000,176902.0312,0.7426,0.1592,0.0992
3,77008.7344,23816577024.0000,154326.2031,0.8451,0.1560,0.1037
4,87897.0156,29909733376.0000,172944.3125,0.8348,0.1601,0.1045
5,71732.0938,21894539264.0000,147968.0312,0.8500,0.1310,0.0874
6,84131.4766,21271197696.0000,145846.4844,0.7691,0.1570,0.1071
7,95130.8594,124909772800.0000,353425.7812,0.5587,0.1702,0.1004
8,73139.5625,20251367424.0000,142307.2969,0.7326,0.1481,0.0950


Fitting 10 folds for each of 10 candidates, totalling 100 fits


<bound method XGBModel.get_params of XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9,
             data=          cena  ulica  pokoje  powierzchnia  piętro  liczba pięter  \
0     899000.0    384       4         88.50       0              0   
1     850000.0    699       3         76.36       1              3   
2     539000.0    182       4         67.50       1              1   
3     620000.0    125       1         29.62       4              7   
4     806000.0    315       3         63.26       4              6   
...        ...    ...     ...           ...     ...            ...   
7112  542459.5    254       3         51.91       2              2   
7113  569000.0    478       2         61.70       1              3   
7114  6...
             eval_metric=None, feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,

In [6]:
final_model = predict_model(tuned_model, data=test_data)

# Sample of y_test (actual values) and y_pred (predicted values)
final_model['ER'] = (final_model['cena'] - final_model['prediction_label'])/1e3
final_model

,ulica,pokoje,powierzchnia,piętro,liczba pięter,stan wykonczenia,balkon/ogrod/taras,miejsce parkingowe,rok budowy,cena,prediction_label,ER
7117,588,3,75.000000,10,17,2,False,False,1969.0,2080000.0,1.114107e+06,965.892639
7118,125,3,69.000000,1,4,0,True,False,1969.0,700000.0,8.130978e+05,-113.097809
7119,636,2,40.500000,6,7,1,True,True,1969.0,722844.0,7.050719e+05,17.772062
7120,260,2,40.200001,0,7,0,True,True,1969.0,630000.0,6.609258e+05,-30.925751
7121,211,3,60.299999,4,4,1,True,True,1969.0,757537.0,7.051917e+05,52.345314
...,...,...,...,...,...,...,...,...,...,...,...,...
9485,137,3,52.000000,3,4,2,True,True,1969.0,624000.0,6.942277e+05,-70.227684
9486,125,3,102.650002,9,9,2,True,True,1969.0,2400000.0,1.566742e+06,833.257507
9487,199,3,100.089996,7,7,1,False,True,1969.0,1351349.0,1.320304e+06,31.045500
9488,39,2,40.540001,1,2,0,True,True,1969.0,419589.0,4.396123e+05,-20.023281


In [7]:
save_model(tuned_model, 'model1')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(include=['ulica', 'pokoje', 'powierzchnia',
                                              'piętro', 'liczba pięter',
                                              'stan wykonczenia', 'rok budowy'],
                                     transformer=SimpleImputer())),
                 ('categorical_imputer',
                  TransformerWrapper(include=[],
                                     transformer=SimpleImputer(strategy='most_frequent'))),
                 ('clean_column_names',
                  Transforme...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.05,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
    